## Adım 1 : Veri setinin hazırlanması

* Bir movie veri seti kullanacağız. Bu veri seti üzerinden bir pivot table oluşturulacaktır. Bu pivot tablosunun değerlerinde "rating",  indexinde "userId", sütunda ise "title" yer alacaktır.

In [1]:
import pandas as pd

In [2]:
movie = pd.read_csv("movie.csv")
rating = pd.read_csv("rating.csv")
df = movie.merge(rating, how="left", on="movieId")
print(df.head())

   movieId             title                                       genres  \
0        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
1        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
2        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
3        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   
4        1  Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy   

   userId  rating            timestamp  
0     3.0     4.0  1999-12-11 13:36:47  
1     6.0     5.0  1997-03-13 17:50:52  
2     8.0     4.0  1996-06-05 13:37:51  
3    10.0     4.0  1999-11-25 02:44:47  
4    11.0     4.5  2009-01-02 01:13:41  


In [8]:
comment_counts = pd.DataFrame(df["title"].value_counts())
print(comment_counts) # Filmlerin oylanma sayıları

                                           count
title                                           
Pulp Fiction (1994)                        67310
Forrest Gump (1994)                        66172
Shawshank Redemption, The (1994)           63366
Silence of the Lambs, The (1991)           63299
Jurassic Park (1993)                       59715
...                                          ...
Rapture (Arrebato) (1980)                      1
Education of Mohammad Hussein, The (2013)      1
Satanas (2007)                                 1
Psychosis (2010)                               1
Innocence (2014)                               1

[27262 rows x 1 columns]


In [14]:
rare_movies = comment_counts[comment_counts["count"] <= 10000].index
common_movies = df[~df["title"].isin(rare_movies)]
user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
print(user_movie_df)

title     10 Things I Hate About You (1999)  12 Angry Men (1957)  \
userId                                                             
1.0                                     NaN                  NaN   
2.0                                     NaN                  NaN   
3.0                                     NaN                  NaN   
4.0                                     NaN                  NaN   
5.0                                     NaN                  NaN   
...                                     ...                  ...   
138489.0                                NaN                  4.5   
138490.0                                NaN                  NaN   
138491.0                                NaN                  NaN   
138492.0                                NaN                  NaN   
138493.0                                NaN                  4.0   

title     2001: A Space Odyssey (1968)  28 Days Later (2002)  300 (2007)  \
userId                                 

* Oylanma sayısı 10000'den fazla olan filmleri aldık ve bir pivot table oluşturduk.
    - Value kısmında : Rating
    - Satırlarda : Index
    - Sütunlarda : Film isimleri

## Adım 2 : Öneri yapılacak kullanıcının izlediği filmlerin belirlenmesi

### Bir rastgele kullanıcı belirleyip, tavsiye sistemimizi oluşturmaya başlayabiliriz.

In [23]:
random_user = int(pd.Series(user_movie_df.index).sample(1, random_state=35).values) # Rastgele kullanıcı (8362) 
movies_watched = random_user_df.columns[random_user_df.notna().any()].tolist() # İzlenmeyen filmleri çıkartıyoruz

len(movies_watched)

50

* Başlangıçta 462 tane filmimiz var iken, 50 filme düşürdük. Yani, kullanıcımız 50 tane film izlemiş diyebiliriz.

## Adım 3: Aynı Filmleri İzleyen Diğer Kullanıcıların Verisine ve Id'lerine Erişmek

In [28]:
movies_watched_df = user_movie_df[movies_watched]
user_movie_count = movies_watched_df.T.notnull().sum() 
user_movie_count = user_movie_count.reset_index() 
user_movie_count.columns = ["userId", "movie_count"]
user_movie_count[user_movie_count["movie_count"] > 40].sort_values("movie_count", ascending=False) 
user_movie_count[user_movie_count["movie_count"] == 50].count() 
users_same_movies = user_movie_count[user_movie_count["movie_count"] > 40]["userId"]
print(users_same_movies)

28            29.0
90            91.0
115          116.0
155          156.0
293          294.0
            ...   
137335    138168.0
137374    138208.0
137444    138279.0
137647    138483.0
137648    138484.0
Name: userId, Length: 2565, dtype: float64


* İlk başta, tüm kullanıcılar için, yalnızca random kullanıcının izlediği filmleri içeren bir DataFrame oluşturuldu.
* DataFrame transpoze edilerek (satır-sütun değişimi) her kullanıcının random kullanıcının izlediği filmlerden kaç tanesini oyladığı hesaplandı.
* Seçilen kullanıcı ile ortak 40 film izleyen kullanıcılar filtrelendi ve veri seti güncellendi.

## Adım 4: Öneri Yapılacak Kullanıcı ile En Benzer Davranışlı Kullanıcıların Belirlenmesi

In [30]:
final_df = pd.concat([movies_watched_df[movies_watched_df.index.isin(users_same_movies)],
                      random_user_df[movies_watched]])
print(final_df)

# Seçilen benzer kullanıcıların oylama verileri ile random kullanıcının verileri, aynı filmler (movies_watched) üzerinden birleştirilir.

title     Ace Ventura: Pet Detective (1994)  \
userId                                        
29.0                                    3.0   
91.0                                    2.5   
116.0                                   3.5   
156.0                                   3.0   
294.0                                   2.5   
...                                     ...   
138208.0                                2.0   
138279.0                                3.0   
138483.0                                1.0   
138484.0                                3.0   
8362.0                                  1.0   

title     Ace Ventura: When Nature Calls (1995)  Addams Family Values (1993)  \
userId                                                                         
29.0                                        3.0                          3.0   
91.0                                        2.0                          2.5   
116.0                                       2.5                     

In [32]:
print(final_df.index[final_df.index.duplicated()]) # duplicate mevcut.
final_df = final_df[~final_df.index.duplicated(keep='first')]

Index([8362.0], dtype='float64', name='userId')


In [33]:
corr_df = final_df.T.corr().unstack().sort_values().drop_duplicates()    
corr_df = pd.DataFrame(corr_df, columns=["corr"])
corr_df.index.names = ['user_id_1', 'user_id_2']
corr_df = corr_df.reset_index()
print(corr_df)

         user_id_1  user_id_2      corr
0          40505.0    67756.0 -0.751181
1          18893.0    59226.0 -0.703250
2          67756.0    27406.0 -0.695150
3          26386.0    27239.0 -0.684906
4          61858.0   124395.0 -0.680414
...            ...        ...       ...
3259441    46836.0    54548.0  0.946763
3259442    68063.0    32344.0  0.969623
3259443   137686.0    39619.0  0.971376
3259444       29.0       29.0  1.000000
3259445     1376.0     3743.0       NaN

[3259446 rows x 3 columns]


* Kullanıcılar arasında bir korelasyon hesaplandı ve bu değerler ile bir df oluşturuldu.

### Bu korelasyon hesaplamamızdaki amaç, seçilen kullanıcı ile en çok korelasyona sahip kullanıcıları belirlemektir.

In [34]:
top_users = corr_df[(corr_df["user_id_1"] == random_user) & (corr_df["corr"] >= 0.65)][
    ["user_id_2", "corr"]].reset_index(drop=True) # Korelasyon değeri 0.65 üzeri kullanıcıları seçeceğiz.

top_users = top_users.sort_values(by='corr', ascending=False)
top_users.rename(columns={"user_id_2": "userId"}, inplace=True)

# Hangi kullanıcının kaç puan verdiği hakkında bir bilgimiz yok. Bu yüzden rating veri setimizi tekrardan kullanıyoruz.
top_users_ratings = top_users.merge(rating[["userId", "movieId", "rating"]], how='inner')
print(top_users_ratings)

        userId      corr  movieId  rating
0      58648.0  0.743338        1     5.0
1      58648.0  0.743338        2     3.0
2      58648.0  0.743338        6     4.5
3      58648.0  0.743338        7     3.5
4      58648.0  0.743338       10     3.0
...        ...       ...      ...     ...
32492  66763.0  0.650556    91529     4.0
32493  66763.0  0.650556    91658     3.5
32494  66763.0  0.650556    92259     4.0
32495  66763.0  0.650556    96728     3.5
32496  66763.0  0.650556    98154     4.5

[32497 rows x 4 columns]


## Adım 5: Weighted Average Recommendation Score'un Hesaplanması 

* top_users_ratings çıktısında görüldüğü üzere, korelasyon değerlerini ve ratingleri görebiliyoruz. Şimdi, buradaki tavsiye sıralaması nasıl yapılacak ?
    - Sadece rating ile sıralayacak olursak, korelasyon değerlerini hiçe saymış olacağız. 
    - Sadece korelasyon ile sıralama yapacaksak, rating puanlarını hesaba katmamış olacağız.

* Rating hemde Korelasyonu beraber kullanıp sıralama yapmamız daha sağlıklı olacaktır.

* Bunun için, "weighted_rating" adında bir sütun oluşturacağız. Seçilen kullanıcıya ise, userId groupby'ını alarak weighted_rating'in ortalamasını alıp o şekilde sıralama sunacağız

In [37]:
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['rating']
top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

recommendation_df = top_users_ratings.groupby('movieId').agg({"weighted_rating": "mean"})

recommendation_df = recommendation_df.reset_index() # 7546 tane kullanıcı var. Bunları weighted_rating'e göre eleyeceğiz.

movies_to_be_recommend = recommendation_df[recommendation_df["weighted_rating"] > 3.5].sort_values("weighted_rating", ascending=False)
print(movies_to_be_recommend.merge(movie[["movieId", "title"]]))

   movieId  weighted_rating                                         title
0     4612         3.571748  Jesus of Montreal (Jésus de Montréal) (1989)
1     7478         3.571748                   Swimming to Cambodia (1987)
2     3038         3.544738                   Face in the Crowd, A (1957)
3    78986         3.544738                 Salvador (Puig Antich) (2006)
4   106487         3.544738       Hunger Games: Catching Fire, The (2013)
